In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import itertools
import tensorflow as tf
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout, Dense, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping



In [2]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')



for df in [train, test]:
    df.drop(columns=['Soil_Type7', 'Soil_Type15','Soil_Type1'],inplace=True)
    
#feature engineering
'''
train['total_distance_to_water'] = np.sqrt((train['Horizontal_Distance_To_Hydrology'])**2 + (train['Vertical_Distance_To_Hydrology'])**2)
train = train.drop(['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology'], axis=1)

test['total_distance_to_water'] = np.sqrt((test['Horizontal_Distance_To_Hydrology'])**2 + (test['Vertical_Distance_To_Hydrology'])**2)
test = test.drop(['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology'], axis=1)
#train['total_shade'] = train['Hillshade_9am'] + train['Hillshade_Noon'] + train['Hillshade_3pm']
train['mean_shade'] = (train['Hillshade_9am'] + train['Hillshade_Noon'] + train['Hillshade_3pm'])/3
train= train.drop(['Hillshade_9am','Hillshade_Noon', 'Hillshade_3pm'],axis=1)

test['mean_shade'] = (test['Hillshade_9am'] + test['Hillshade_Noon'] + test['Hillshade_3pm'])/3
test= test.drop(['Hillshade_9am','Hillshade_Noon', 'Hillshade_3pm'],axis=1)
'''
train = train[train.Cover_Type != 5]
X = train.drop(['Cover_Type','Id'], axis=1)
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(train.Cover_Type))
X_test = test.drop('Id', axis=1)



In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16',
                'float32', 'float64']
    for col in df.columns:
        if df[col].dtype=='bool':
            df[col] = df[col].astype(int)
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            #change int type to lowest poss
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

X = reduce_mem_usage(X)
X_test = reduce_mem_usage(X_test)

Mem. usage decreased to 259.40 Mb (83.7% reduction)
Mem. usage decreased to 57.22 Mb (85.3% reduction)


In [4]:
score_list, test_pred_list, history_list = [], [], []

EPOCHS = 90
VERBOSE = 1
SINGLE_FOLD = False   
BATCH_SIZE = 800
FOLDS = 10
RUNS = 1  # should be 1. increase the number of runs only if you want see how the result depends on the random seed

def my_model(X):

    
    model = Sequential()
    model.add(InputLayer(input_shape=(X.shape[-1])))
    model.add(Dense(128, activation='selu'))
    model.add(BatchNormalization())
   # model.add(Dropout(.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    # model.add(Dropout(.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    # model.add(Dropout(.1))
    model.add(Dense(6, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

    return model

np.random.seed(1)
tf.random.set_seed(1)

for run in range(RUNS):
    kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=1)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y=y)):
        X_tr = X.iloc[train_idx]
        X_va = X.iloc[val_idx]
        y_tr = y.iloc[train_idx]
        y_va = y.iloc[val_idx]

        scaler = StandardScaler()

        X_tr = pd.DataFrame(scaler.fit_transform(X_tr))
        X_va = pd.DataFrame(scaler.transform(X_va))
        
        model = my_model(X_tr)

        #define callbacks
        lr = ReduceLROnPlateau(monitor='val_loss', factor=.5, patience=5, verbose=VERBOSE)

        es = EarlyStopping(monitor='val_loss', patience=10, verbose=VERBOSE, mode='min', restore_best_weights=True)

#train and save model
        history = model.fit(X_tr, y_tr, validation_data=(X_va, y_va), epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_batch_size= len(X_va), callbacks=[lr,es], shuffle=True)

        history_list.append(history.history)
        model.save(f'model{run}.{fold}')

        #inference for validation after last epoch of fold
        y_va_pred = model.predict(X_va, batch_size=len(X_va))
        y_va_pred = np.argmax(y_va_pred, axis=1)

        #evaluation
        accuracy = accuracy_score(y_va, y_va_pred)

        print(f'Fold {run}.{fold} : Accuracy: {accuracy:.5f}')

        #test predicts
        test_pred_list.append(model.predict(scaler.transform(X_test), batch_size=BATCH_SIZE))

sub = pd.DataFrame()
sub['Id'] = test['Id']
sub['Cover_Type'] = le.inverse_transform(np.argmax(sum(test_pred_list), axis=1))
sub.to_csv('submission.csv', index=False)

2021-12-15 13:09:53.514468: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 13:09:53.612894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 13:09:53.613613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 13:09:53.614711: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/90
4500/4500 [==============================] - 21s 4ms/step - loss: 0.1460 - acc: 0.9425 - val_loss: 0.1067 - val_acc: 0.9539
Epoch 2/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.1064 - acc: 0.9537 - val_loss: 0.0983 - val_acc: 0.9570
Epoch 3/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.1009 - acc: 0.9555 - val_loss: 0.0938 - val_acc: 0.9583
Epoch 4/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.0976 - acc: 0.9564 - val_loss: 0.0917 - val_acc: 0.9588
Epoch 5/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0950 - acc: 0.9573 - val_loss: 0.0904 - val_acc: 0.9592
Epoch 6/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.0935 - acc: 0.9578 - val_loss: 0.0899 - val_acc: 0.9592
Epoch 7/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0919 - acc: 0.9583 - val_loss: 0.0881 - val_acc: 0.9601
Epoch 8/90
4500/4500 [==============================] -

2021-12-15 13:35:53.590950: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Fold 0.0 : Accuracy: 0.96220


2021-12-15 13:36:04.752735: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1468799592 exceeds 10% of free system memory.
2021-12-15 13:36:06.197411: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1468799592 exceeds 10% of free system memory.


Epoch 1/90
4500/4500 [==============================] - 24s 5ms/step - loss: 0.1447 - acc: 0.9433 - val_loss: 0.1048 - val_acc: 0.9548
Epoch 2/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.1069 - acc: 0.9537 - val_loss: 0.1020 - val_acc: 0.9554
Epoch 3/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.1011 - acc: 0.9555 - val_loss: 0.0932 - val_acc: 0.9583
Epoch 4/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0978 - acc: 0.9566 - val_loss: 0.0931 - val_acc: 0.9582
Epoch 5/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0953 - acc: 0.9573 - val_loss: 0.0924 - val_acc: 0.9584
Epoch 6/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0934 - acc: 0.9579 - val_loss: 0.0885 - val_acc: 0.9597
Epoch 7/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0922 - acc: 0.9583 - val_loss: 0.0881 - val_acc: 0.9597
Epoch 8/90
4500/4500 [==============================] -

2021-12-15 14:02:40.737964: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1468799592 exceeds 10% of free system memory.


Epoch 1/90
4500/4500 [==============================] - 20s 4ms/step - loss: 0.1444 - acc: 0.9431 - val_loss: 0.1057 - val_acc: 0.9546
Epoch 2/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.1065 - acc: 0.9537 - val_loss: 0.0972 - val_acc: 0.9572
Epoch 3/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.1007 - acc: 0.9555 - val_loss: 0.0986 - val_acc: 0.9567
Epoch 4/90
4500/4500 [==============================] - 19s 4ms/step - loss: 0.0976 - acc: 0.9566 - val_loss: 0.0917 - val_acc: 0.9590
Epoch 5/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0950 - acc: 0.9574 - val_loss: 0.0893 - val_acc: 0.9599
Epoch 6/90
4500/4500 [==============================] - 18s 4ms/step - loss: 0.0934 - acc: 0.9580 - val_loss: 0.0895 - val_acc: 0.9594
Epoch 7/90
4500/4500 [==============================] - 24s 5ms/step - loss: 0.0920 - acc: 0.9583 - val_loss: 0.0862 - val_acc: 0.9603
Epoch 8/90
4500/4500 [==============================] -